In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import cross_val_score, learning_curve
from xgboost import XGBClassifier
import shap
from lime import lime_text
import eli5
from joblib import load


ModuleNotFoundError: No module named 'lime'

In [ ]:

# 1. Cargar el mejor modelo y los datos de prueba
best_model = load('../models/best_model.joblib')
X_test = pd.read_csv('../data/processed/X_test.csv')
y_test = pd.read_csv('../data/processed/y_test.csv')

# 2. Evaluación detallada del modelo
def detailed_evaluation(model, X, y):
    y_pred = model.predict(X)
    y_prob = model.predict_proba(X)[:, 1]

    # Matriz de confusión
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Matriz de Confusión')
    plt.ylabel('Valor Real')
    plt.xlabel('Predicción')
    plt.show()

    # Reporte de clasificación
    print(classification_report(y, y_pred))

    # Curva ROC
    fpr, tpr, _ = roc_curve(y, y_prob)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Tasa de Falsos Positivos')
    plt.ylabel('Tasa de Verdaderos Positivos')
    plt.title('Curva Característica Operativa del Receptor (ROC)')
    plt.legend(loc="lower right")
    plt.show()

detailed_evaluation(best_model, X_test, y_test)

# 3. Validación cruzada
cv_scores = cross_val_score(best_model, X_test, y_test, cv=5)
print(f"Puntuaciones de validación cruzada: {cv_scores}")
print(f"Media de validación cruzada: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

# 4. Curva de aprendizaje
train_sizes, train_scores, test_scores = learning_curve(
    best_model, X_test, y_test, cv=5, n_jobs=-1, 
    train_sizes=np.linspace(.1, 1.0, 5))

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 8))
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Puntuación de entrenamiento")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Puntuación de validación cruzada")
plt.xlabel("Tamaño del conjunto de entrenamiento")
plt.ylabel("Puntuación")
plt.title("Curva de Aprendizaje")
plt.legend(loc="best")
plt.show()

# 5. Interpretabilidad del modelo
# SHAP (SHapley Additive exPlanations)
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)

plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test, plot_type="bar")
plt.show()

# LIME (Local Interpretable Model-agnostic Explanations)
explainer = lime_text.LimeTextExplainer(class_names=['No tóxico', 'Tóxico'])
idx = 0  # índice de la instancia a explicar
exp = explainer.explain_instance(X_test.iloc[idx], best_model.predict_proba, num_features=10)
exp.show_in_notebook(text=True)

# ELI5 (Explain Like I'm 5)
eli5.show_weights(best_model, feature_names=X_test.columns.tolist())

# 6. Análisis de errores
y_pred = best_model.predict(X_test)
errors = X_test[y_test != y_pred]
print("Ejemplos de predicciones erróneas:")
for idx, row in errors.head().iterrows():
    print(f"Texto: {row['text']}")
    print(f"Etiqueta real: {y_test.iloc[idx]}")
    print(f"Predicción: {y_pred[idx]}")
    print("---")

# 7. Pruebas con nuevos ejemplos
new_texts = [
    "Eres una persona maravillosa",
    "Ojalá te mueras, imbécil",
    "Me encanta tu trabajo, sigue así"
]

for text in new_texts:
    prob = best_model.predict_proba([text])[0]
    print(f"Texto: {text}")
    print(f"Probabilidad de ser tóxico: {prob[1]:.2f}")
    print("---")

print("Evaluación del modelo completada.")
